In [1]:
import keras
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential,load_model
import tensorflow.keras.backend as K
from commai_spd_chl_keras_datagen import commai_speed_chl_gen
from tensorflow.keras.layers import Dense,Activation,Lambda,GRU,BatchNormalization,Activation,Flatten,Conv2D,MaxPooling2D,Reshape,Dropout

Using TensorFlow backend.


In [2]:
tf.config.list_physical_devices ()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
tf.keras.backend.image_data_format()

'channels_last'

In [4]:
seq = Sequential()

#should I trust this guy?https://www.kdnuggets.com/2018/09/dropout-convolutional-networks.html

seq.add(Conv2D(filters=32,kernel_initializer='he_normal', kernel_size=(4,4),padding='same',input_shape=(480,640,3)))
seq.add(MaxPooling2D(pool_size=(4,4)))
seq.add(Activation("relu"))
seq.add(BatchNormalization())

seq.add(Conv2D(filters=32, kernel_size=(4,4),kernel_initializer='he_normal',padding='same'))
seq.add(MaxPooling2D(pool_size=(4,4)))
seq.add(Activation("relu"))
seq.add(BatchNormalization())

seq.add(Conv2D(filters=64, kernel_size=(4,4),kernel_initializer='he_normal',padding='same'))
seq.add(MaxPooling2D(pool_size=(4,4)))
seq.add(Activation("relu"))
seq.add(BatchNormalization())

seq.add(Conv2D(filters=64, kernel_size=(4,4),kernel_initializer='he_normal',padding='same'))
seq.add(MaxPooling2D(pool_size=(4,4)))
seq.add(Activation("relu"))
seq.add(BatchNormalization())

seq.add(Flatten())

seq.add(Dense(192,kernel_initializer='he_normal'))
seq.add(Activation("relu"))
seq.add(BatchNormalization())
seq.add(Dropout(.3)) # throw away 30%

seq.add(Dense(50,kernel_initializer='he_normal'))
seq.add(Activation("relu"))
seq.add(Dropout(.2))

seq.add(Dense(1,kernel_initializer='he_normal'))

seq.compile(loss='mse', optimizer='adam')
seq.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 480, 640, 32)      1568      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 120, 160, 32)      0         
_________________________________________________________________
activation (Activation)      (None, 120, 160, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 120, 160, 32)      128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 120, 160, 32)      16416     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 40, 32)        0         
_________________________________________________________________
activation_1 (Activation)    (None, 30, 40, 32)        0

In [5]:
commai_speed_chl_gen().build_video_folder()

  0%|          | 8/20400 [00:00<04:35, 74.01it/s]

(480, 640)
(32, 3, 480, 640)
(32, 3, 1)
building video Database


100%|██████████| 20400/20400 [07:35<00:00, 44.75it/s]


In [4]:
batch_size=45
n_e=60
n_frames_train=17000
n_frames_test=1900
n_frames_valid=1500

train_idx,test_idx,valid_idx=commai_speed_chl_gen(n_frames_valid=n_frames_valid,frames_per_sample=2,mode="optical_flow_dense",
                    n_frames_test=n_frames_test,n_frames_train=n_frames_train,batch_size=batch_size).get_indexes()

load=np.load(open("idxes_45bs_opflowdense_17k.np","rb"))
train_idx=load["train_idx"]
test_idx=load["test_idx"]
valid_idx=load["valid_idx"]

train_gen=commai_speed_chl_gen(frame_idx=train_idx,batch_size=batch_size,mode="optical_flow_dense",frames_per_sample=2)
test_gen=commai_speed_chl_gen(frame_idx=test_idx,batch_size=batch_size,mode="optical_flow_dense",frames_per_sample=2)
valid_gen=commai_speed_chl_gen(frame_idx=valid_idx,batch_size=batch_size,mode="optical_flow_dense",frames_per_sample=2)

#np.savez_compressed(open("idxes_45bs_opflowdense_17k.np","wb+"),train_idx=train_idx,test_idx=test_idx,valid_idx=valid_idx)



(480, 640, 3)
(45, 480, 640, 3)
(45, 1)
train_samples: 8500 
test_samples:950 
valid_samples: 750
(480, 640, 3)
(45, 480, 640, 3)
(45, 1)
(480, 640, 3)
(45, 480, 640, 3)
(45, 1)
(480, 640, 3)
(45, 480, 640, 3)
(45, 1)


In [ ]:
ie=0
filepath="STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-17k-{epoch:02d}-{val_loss:.2f}-{loss:.2f}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False)

tb=tf.keras.callbacks.TensorBoard(
    log_dir='STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-17k', histogram_freq=1, write_graph=True, write_images=False,
    update_freq=10, profile_batch=0, embeddings_freq=0
)

cbs=[checkpoint,tb]
hst=seq.fit_generator(train_gen, epochs=n_e,workers=25,max_queue_size=85,
        use_multiprocessing=True,validation_data=test_gen,callbacks=cbs,verbose=1,shuffle=True, initial_epoch=ie)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/60
188/188 [==============================] - ETA: 0s - loss: 66.2430WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00001: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-17k-01-33.52-66.24.hdf5
188/188 [==============================] - 257s 1s/step - loss: 66.2430 - val_loss: 33.5168
Epoch 2/60
188/188 [==============================] - ETA: 0s - loss: 39.1821WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00002: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-17k-02-25.50-39.18.hdf5
188/188 [==============================] - 280s 1s/step - loss: 39.1821 - val_loss: 25.5031
Epoch 3/60
188/188


Epoch 00008: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-17k-08-10.23-15.70.hdf5
188/188 [==============================] - 270s 1s/step - loss: 15.7027 - val_loss: 10.2328
Epoch 9/60
188/188 [==============================] - ETA: 0s - loss: 14.2263WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00009: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-17k-09-8.08-14.23.hdf5
188/188 [==============================] - 293s 2s/step - loss: 14.2263 - val_loss: 8.0759
Epoch 10/60
188/188 [==============================] - ETA: 0s - loss: 12.4784WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00010: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-17

188/188 [==============================] - 280s 1s/step - loss: 8.9058 - val_loss: 6.8769
Epoch 17/60
188/188 [==============================] - ETA: 0s - loss: 8.6828WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00017: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-17k-17-7.64-8.68.hdf5
188/188 [==============================] - 289s 2s/step - loss: 8.6828 - val_loss: 7.6401
Epoch 18/60
188/188 [==============================] - ETA: 0s - loss: 8.4335WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00018: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-17k-18-8.78-8.43.hdf5
188/188 [==============================] - 269s 1s/step - loss: 8.4335 - val_loss: 8.7762
Epoch 19/60


Epoch 25/60
188/188 [==============================] - ETA: 0s - loss: 6.8257WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00025: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-17k-25-6.49-6.83.hdf5
188/188 [==============================] - 294s 2s/step - loss: 6.8257 - val_loss: 6.4942
Epoch 26/60
188/188 [==============================] - ETA: 0s - loss: 6.6656WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00026: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-17k-26-7.33-6.67.hdf5
188/188 [==============================] - 279s 1s/step - loss: 6.6656 - val_loss: 7.3331
Epoch 27/60
188/188 [==============================] - ETA: 0s - loss: 6.6106WARNING:tensorflow:multip

188/188 [==============================] - ETA: 0s - loss: 5.9918WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00033: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-17k-33-8.30-5.99.hdf5
188/188 [==============================] - 268s 1s/step - loss: 5.9918 - val_loss: 8.2974
Epoch 34/60
188/188 [==============================] - ETA: 0s - loss: 5.8119WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00034: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-17k-34-5.82-5.81.hdf5
188/188 [==============================] - 298s 2s/step - loss: 5.8119 - val_loss: 5.8238
Epoch 35/60
 24/188 [==>...........................] - ETA: 2:41 - loss: 5.7420

In [6]:
#34 Epoches
seq=load_model("STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-17k-34-5.82-5.81.hdf5")
seq.evaluate_generator(valid_gen,n_frames_valid/batch_size/2, max_queue_size = 80,workers = 10,verbose=2)

Instructions for updating:
Please use Model.evaluate, which supports generators.
17/16 - 15s - loss: 6.6374


6.637359619140625

In [7]:
#27 Epoches
seq=load_model("STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-17k-27-6.84-6.61.hdf5")
seq.evaluate_generator(valid_gen,n_frames_valid/batch_size/2, max_queue_size = 80,workers = 10,verbose=2)

17/16 - 17s - loss: 7.8507


7.85069465637207